In [1]:
import os
import json
import javalang
from openai import OpenAI

In [2]:
template = """
You are a programming assistant specialized in analyzing Java code. Your task is to provide a concise yet comprehensive summary of a given Java method. The summary should be written as a single, cohesive paragraph that includes the following:

1. **Purpose:** Briefly describe what the method is designed to accomplish.
2. **Parameters:** Explain the purpose of each parameter, including their types, and how they are used in the method.
3. **Return Value:** Describe what the method returns, including its type and significance.
4. **Core Logic:** Summarize the key functionality and major steps the method performs.
5. **Exceptions:** Mention any exceptions that the method might throw and under what conditions.
6. **Context (if applicable):** Explain how the method interacts with other components or fits into the larger codebase.

The paragraph should use plain English, avoiding excessive technical jargon, while ensuring no key details are omitted. Use the following Java method to generate the summary:
"""

In [3]:
def generate_summary_using_llm(method):
    my_api_key = ""
    try:
        with open("/Users/abdulrafay/Desktop/Research Project/parser/Key/my_key") as keyfile:
            my_api_key = keyfile.readline().strip()
    except FileNotFoundError:
        print("Error: The file 'my_key' was not found. Please make sure the file exists and contains your API key.")


    client = OpenAI(base_url="https://llm.scads.ai/v1",api_key=my_api_key)

    for model in client.models.list().data:
        model_name = model.id

        if "llama" in model_name:
            break
        
    request = template + method

    try:
        response = client.chat.completions.create(
            messages=[
                {"role":"user","content":request}
            ],
            model=model_name,
            temperature = 0.1, 
            max_tokens = 2048
        )

        res = response.choices[0].message.content

        return res

    except json.JSONDecodeError as e:
        print("JSON decode error:", e)
        print("Response content might not be valid JSON:", response.choices[0].message.content if response.choices else "No content")

        return 

In [4]:
def extract_code_with_brackets(entity, lines):
    if entity.position:
        start_line = entity.position[0] - 1 
        
        open_brackets = 0
        end_line = start_line
        for i in range(start_line, len(lines)):
            open_brackets += lines[i].count('{')
            open_brackets -= lines[i].count('}')
            if open_brackets == 0:
                end_line = i
                break

        return "\n".join(lines[start_line:end_line + 1])
    return "<Code could not be extracted>"

In [5]:
def get_imports_used_in_method(method_body, imports):
    used_imports = []
    for _import in imports:
        import_name = _import.path.split('.')[-1] 
        if any(import_name in line for line in method_body.splitlines()):
            used_imports.append(_import.path)
    return used_imports

In [6]:
def get_package_name(tree, repo_structure):
    repo_structure['package'] = tree.package.name

In [7]:
def get_imports(tree, repo_structure):
    import_lst = []

    for imp in tree.imports:
        import_lst.append(imp.path)
    
    repo_structure['imports'] = import_lst

In [8]:
def get_class_imports(tree, node):
    used_imports = []
    for _import in tree.imports:
        import_name = _import.path.split('.')[-1] 
        if import_name in node.name:
            used_imports.append(_import.path)
    
    return used_imports

In [9]:
def get_mofifiers(node_modifiers):
    modifier_lst = []

    for modi in node_modifiers:
        modifier_lst.append(modi)

    return modifier_lst

In [10]:
def get_annotations(annotations_body):
    annotations_lst = []
    if annotations_body:
        for annotation in annotations_body:
            annotations_lst.append(annotation.name)
            
    return annotations_lst

In [11]:
def get_parameters(parameters_body):
    parameters_lst = []
    for param in parameters_body:
        param_type = getattr(param.type, 'name', 'Unknown')
        data = {
            "name": param.name,
            "type": param_type
        }
        parameters_lst.append(data)

    return parameters_lst

In [12]:
def get_class_variables(node):
    variable_lst = []
    for body_item in node.body:
        if isinstance(body_item, javalang.tree.FieldDeclaration):
            for declarator in body_item.declarators:
                field_type = getattr(body_item.type, 'name', 'Unknown')
                data = {
                    "name": declarator.name,
                    "type": field_type,
                    "modifiers": get_mofifiers(body_item.modifiers)
                }
                variable_lst.append(data)
    return variable_lst

In [13]:
def get_constructors(node, lines, tree):
    constructor_lst = []
    for body_item in node.body:
        if isinstance(body_item, javalang.tree.ConstructorDeclaration):
            full_constructor_code = extract_code_with_brackets(body_item, lines)

            used_imports = get_imports_used_in_method(full_constructor_code, tree.imports)

            data = {
                "name": body_item.name,
                "modifiers": get_mofifiers(body_item.modifiers),
                "annotations": get_annotations(body_item.annotations),
                "parameters": get_parameters(body_item.parameters),
                "used_imports": used_imports
            }
            constructor_lst.append(data)

    return constructor_lst
    

In [14]:
def get_throws(method):   

    if method:
        return method
    else:
        return []

In [15]:
def get_methods(node, lines, tree):
    method_lst = []
    for body_item in node.body:
        if isinstance(body_item, javalang.tree.MethodDeclaration):

            full_method_code = extract_code_with_brackets(body_item, lines) 

            # print(full_method_code)
            # print("===="*10)

            used_imports = get_imports_used_in_method(full_method_code, tree.imports)
            data = {
                "name": body_item.name,
                "modifiers": get_mofifiers(body_item.modifiers),
                "return_type": body_item.return_type.name if body_item.return_type else 'void',
                "annotations": get_annotations(body_item.annotations),
                "parameters": get_parameters(body_item.parameters),
                "used_imports": used_imports,
                "throws": get_throws(body_item.throws)
            }
            method_lst.append(data)
            
    return method_lst

In [16]:
def get_class(node, tree, lines, repo_structure):
    nested_lst = []
    if 'classes' not in repo_structure:
        repo_structure['classes'] = []

    for inner_node in node.body:
        if isinstance(inner_node, javalang.tree.ClassDeclaration):
            nested_lst.append(inner_node.name)

    class_details = {
        'name': node.name,
        "modifiers": get_mofifiers(node.modifiers),
        "extends": node.extends.name if node.extends else None,
        "implements": [iface.name for iface in node.implements] if node.implements else [],
        "nested_classes": nested_lst,
        "used_imports": get_class_imports(tree, node),
        "variables": get_class_variables(node),
        "constructors": get_constructors(node, lines, tree),
        "methods": get_methods(node, lines, tree)
    }

    repo_structure['classes'].append(class_details) 

In [17]:
def read_java_file(file_path):
    try:
        with open(file_path, 'r', encoding='ISO-8859-1') as file:
            content = file.read()

        return content
    
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' does not exist.")
        return None
    
    except IOError:
        print(f"Error: Could not read the file '{file_path}'.")
        return None

In [18]:
def drive(tree, lines, repo_structure):
    for _, node in tree:
        if isinstance(node, javalang.tree.ClassDeclaration):
            get_package_name(tree, repo_structure)
            get_imports(tree, repo_structure)
            get_class(node, tree, lines ,repo_structure)

        elif isinstance(node, javalang.tree.InterfaceDeclaration):
            get_package_name(tree, repo_structure)
            get_imports(tree, repo_structure)

In [ ]:
def file_router(path, repo_structure):
    if path.endswith('.png') or path.endswith('.jpg'):
        repo_structure['size'] = str(os.path.getsize(path)) + " bytes"

    elif path.endswith('.java'):
        java_code = read_java_file(path)

        tree = javalang.parse.parse(java_code)
        lines = java_code.splitlines()
        drive(tree, lines, repo_structure)
            

In [20]:
def get_repo_structure(path):
    repo_structure = {}

    for content in os.listdir(path):
        content_path = os.path.join(path, content)

        if content.startswith('.') or content == ".DS_Store":
            continue

        if os.path.isfile(content_path):
            repo_structure[content] = {}
            file_router(content_path, repo_structure[content])

        elif os.path.isdir(content_path):
            repo_structure[content] = get_repo_structure(content_path)

    return repo_structure

In [21]:
path = '/Users/abdulrafay/Desktop/RP/Code_Extraction/acmeair/acmeair-services/src/main/java/com/acmeair/service'

with open("data.json", "w") as json_file:
    json.dump(get_repo_structure(path), json_file, indent=3)

/Users/abdulrafay/Desktop/RP/Code_Extraction/acmeair/acmeair-services/src/main/java/com/acmeair/service/KeyGenerator.java
/Users/abdulrafay/Desktop/RP/Code_Extraction/acmeair/acmeair-services/src/main/java/com/acmeair/service/BookingService.java
/Users/abdulrafay/Desktop/RP/Code_Extraction/acmeair/acmeair-services/src/main/java/com/acmeair/service/FlightService.java
/Users/abdulrafay/Desktop/RP/Code_Extraction/acmeair/acmeair-services/src/main/java/com/acmeair/service/CustomerService.java
